Module 4

In this module we’ll be looking at data from the New York City tree census:
https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh
This data is collected by volunteers across the city, and is meant to catalog information
about every single tree in the city.
Build a dash app for a arborist studying the health of various tree species (as defined by the
variable ‘spc_common’) across each borough (defined by the variable ‘borough’). This
arborist would like to answer the following two questions for each species and in each
borough:

1. What proportion of trees are in good, fair, or poor health according to the ‘health’ variable?
2. Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees?

Please see the accompanying notebook for an introduction and some notes on the Socrata
API.

Deployment: Dash deployment is more complicated than deploying shiny apps, so deployment in this case is optional (and will result in extra credit). You can read instructions on deploying a dash app to heroku here: https://dash.plot.ly/deployment

In [1]:
import pandas as pd
import numpy as np

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is here, and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [2]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


Looks good, but lets take a look at the shape of this data:

In [3]:
trees.shape

(1000, 45)

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset.

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [4]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29,739,4052307,4022210001
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19,973,4101931,4044750045
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34,449,3338310,3028870001
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34,449,3338342,3029250001
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39,165,3025654,3010850052


If we wanted the next 5, we would use this url:

In [5]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,988418.6997,219825.5227,3.0,145.0,1076229.0,1.011310e+09
1,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,988311.1900,219885.2785,3.0,145.0,1076229.0,1.011310e+09
2,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,987769.1163,217157.8561,3.0,133.0,1086093.0,1.010410e+09
3,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,963073.1998,156635.5542,NaN,NaN,NaN,NaN
4,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,992653.7253,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


You can read more about paging using the Socrata API here

In these docs, you'll also see more advanced functions (called SoQL) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using SoQL is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [6]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,spc_common
0,4619,NaN
1,662,silver maple
2,18,pagoda dogwood
3,3917,littleleaf linden
4,12,American larch
...,...,...
128,362,sugar maple
129,170,Cornelian cherry
130,203,white oak
131,88,Kentucky yellowwood


This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

NOTE: One tip in dealing with URLs: you may need to replace spaces with '%20'. I personally just write out the url and then follow the string with a replace:

In [7]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

# Data Preparation



In [12]:
offset = 1000
max_row = 5000

for x in range(0, max_row, offset):
    #print('x is ' + str(x))
    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=1000&$offset=' + str(x) +\
        '&$select=borocode,spc_common,health,steward,count(tree_id)' +\
        '&$group=borocode,spc_common,health,steward').replace(' ', '%20')
    soql_trees = pd.read_json(soql_url)
    if(x==0):
        df = pd.DataFrame(columns=list(soql_trees.columns.values))
    df = df.append(soql_trees)
    #print(df)
    
    df = df.reset_index(drop=True)  

In [43]:
df = df.dropna(axis=0, how='any')#Remove rows that do not have complete data.

In [42]:
df.head(5)

,borocode,spc_common,health,steward,count_tree_id
0,3,London planetree,Good,3or4,644
1,3,Kentucky coffeetree,Good,4orMore,16
2,3,sycamore maple,Poor,None,41
3,5,Chinese tree lilac,Poor,1or2,1
4,5,red maple,Good,None,5094


# Question 1

What proportion of trees are in good, fair, or poor health according to the ‘health’ variable?

In [22]:
df_totals = df.groupby(['borocode', 'spc_common'])['count_tree_id'].sum()
df_total_by_borocode_species_health = df.groupby(['borocode', 'spc_common', 'health'])['count_tree_id'].sum()

In [23]:
df_totals.head(5)

borocode  spc_common            
1         'Schubert' chokecherry     163
          American beech              22
          American elm              1698
          American hophornbeam        84
          American hornbeam           85
Name: count_tree_id, dtype: int64

In [24]:
df_totals = df_totals.reset_index(drop=False)
df_total_by_borocode_species_health = df_total_by_borocode_species_health.reset_index(drop=False)

In [25]:
df_total_by_borocode_species_health.head(5)

,borocode,spc_common,health,count_tree_id
0,1,'Schubert' chokecherry,Fair,40
1,1,'Schubert' chokecherry,Good,111
2,1,'Schubert' chokecherry,Poor,12
3,1,American beech,Fair,4
4,1,American beech,Good,15


In [35]:
df_totals.columns = ['borocode', 'spc_common', 'total_for_species_in_borough']
df_total_by_borocode_species_health.columns = ['borocode', 'spc_common', 'health', 'total']

In [36]:
tree_proportions = pd.merge(df_total_by_borocode_species_health, df_totals, on=['borocode', 'spc_common'])#Combines into one data table

In [37]:
tree_proportions.head(5)

,borocode,spc_common,health,total,total_for_species_in_borough
0,1,'Schubert' chokecherry,Fair,40,163
1,1,'Schubert' chokecherry,Good,111,163
2,1,'Schubert' chokecherry,Poor,12,163
3,1,American beech,Fair,4,22
4,1,American beech,Good,15,22


In [38]:
tree_proportions['ratio'] = tree_proportions['total']/ tree_proportions['total_for_species_in_borough']#Adds ratio to table 

In [41]:
tree_proportions.head(5)

,borocode,spc_common,health,total,total_for_species_in_borough,ratio
0,1,'Schubert' chokecherry,Fair,40,163,0.245399
1,1,'Schubert' chokecherry,Good,111,163,0.680982
2,1,'Schubert' chokecherry,Poor,12,163,0.073620
3,1,American beech,Fair,4,22,0.181818
4,1,American beech,Good,15,22,0.681818


# Question 2

Are stewards(steward activity measured by the ‘steward’ variable) having an impact on the health of trees?

In [44]:
list(df['steward'].unique())

['3or4', '4orMore', 'None', '1or2']

In [45]:
df.head(5)

,borocode,spc_common,health,steward,count_tree_id
0,3,London planetree,Good,3or4,644
1,3,Kentucky coffeetree,Good,4orMore,16
2,3,sycamore maple,Poor,None,41
3,5,Chinese tree lilac,Poor,1or2,1
4,5,red maple,Good,None,5094


In [47]:
df.sort_values(by=['borocode', 'spc_common', 'steward']).head(5)

,borocode,spc_common,health,steward,count_tree_id
620,1,'Schubert' chokecherry,Good,1or2,56
2507,1,'Schubert' chokecherry,Poor,1or2,11
3554,1,'Schubert' chokecherry,Fair,1or2,23
1849,1,'Schubert' chokecherry,Good,3or4,21
2350,1,'Schubert' chokecherry,Fair,3or4,2


In [48]:
df_total_by_steward = df.groupby(['borocode', 'spc_common', 'steward'])['count_tree_id'].sum()
df_total_by_steward = df_total_by_steward.reset_index(drop=False)
df_total_by_steward.columns = ['borocode', 'spc_common', 'steward', 'steward_total']
df_total_by_steward.head(5)

,borocode,spc_common,steward,steward_total
0,1,'Schubert' chokecherry,1or2,90
1,1,'Schubert' chokecherry,3or4,23
2,1,'Schubert' chokecherry,4orMore,3
3,1,'Schubert' chokecherry,None,47
4,1,American beech,1or2,10


In [49]:
df_steward = pd.merge(df, df_total_by_steward, on=['borocode', 'spc_common', 'steward'])
df_steward.head(5)

,borocode,spc_common,health,steward,count_tree_id,steward_total
0,3,London planetree,Good,3or4,644,772
1,3,London planetree,Poor,3or4,18,772
2,3,London planetree,Fair,3or4,110,772
3,3,Kentucky coffeetree,Good,4orMore,16,18
4,3,Kentucky coffeetree,Fair,4orMore,2,18


In [51]:
di = {'Poor':1, 'Fair':2, 'Good':3}
df_steward['health_level'] = df_steward['health'].map(di)
df_steward.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)
df_steward['health_index'] = (df_steward['count_tree_id']/df_steward['steward_total']) * df_steward['health_level']
df_steward.sort_values(by=['borocode', 'spc_common', 'steward']).head(10)
df_overall_health_index = df_steward.groupby(['borocode', 'spc_common', 'steward'])['health_index'].sum()
df_overall_health_index = df_overall_health_index.reset_index(drop=False)
df_overall_health_index.columns = ['borocode', 'spc_common', 'steward', 'overall_health_index']
di2 = {'3or4':3, '4orMore':4, 'None':1, '1or2':2}
df_overall_health_index['steward_level'] = df_overall_health_index['steward'].map(di2)
di3 = { 1:'Manhattan', 2:'Bronx', 3:'Brooklyn', 4:'Queens', 5:'Staten Island'}
df_overall_health_index['borough'] = df_overall_health_index['borocode'].map(di3)
df_overall_health_index['spc_common'] = df_overall_health_index['spc_common'].apply(lambda x: x.title())
df_overall_health_index.head(5)

,borocode,spc_common,steward,overall_health_index,steward_level,borough
0,1,'Schubert' Chokecherry,1or2,2.500000,2,Manhattan
1,1,'Schubert' Chokecherry,3or4,2.913043,3,Manhattan
2,1,'Schubert' Chokecherry,4orMore,3.000000,4,Manhattan
3,1,'Schubert' Chokecherry,None,2.638298,1,Manhattan
4,1,American Beech,1or2,2.700000,2,Manhattan
